#### Finetune the saved models to see whether they forgot things

In [1]:
import os
import sys
sys.path.append("..")

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim as optim

from models import get_model, model_config
from data import data_config
from data.datasets import TwilightDuo
from main import get_device

class ProtendArgs:
    model = "FullyConnectedNetwork"
    #model = "VGG_like"
    dataset = "TwilightDuo"
    model_config = [2048, 512]
    #model_config = [32, 'M', 64, 'M', 128, 'M', 256, 'M', 512, 'M', 1024, 'M', 1024, 'M', 1024]


def load_model(args, fname):
    base_folder = "/scratch/ym2380/saved_models"
    fname = f"{fname}.pth"
    fpath = os.path.join(base_folder, args.model, args.dataset, fname)
    
    model = get_model(args)
    model.load_state_dict(torch.load(fpath))
    
    for param in model.parameters():
        param.requires_grad = False
    for param in model.classifier.parameters():
        param.requires_grad = True
    
    return model

def run_epoch(
    dataloader,
    model,
    loss_fn,
    optimizer,
    train,
    device,
    ):
    if train:
        model.train()
    else:
        network.eval()
    
    total_loss = 0
    correct = 0
    total = len(dataloader.dataset)
    
    with torch.set_grad_enabled(train):
        for batch_idx, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            if model_config[args.model]["input_type"] == "flattened":
                X = X.view(X.size(0), -1)
            
            if train:
                optimizer.zero_grad()
            
            outputs = model(X)
            loss_v = loss_fn(outputs, y)
            total_loss += loss_v.item()
            
            if train:
                loss_v.backward()
                optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions =  (predicted == y).sum().item()
            correct += correct_predictions
    
    return correct / total, total_loss / total

In [2]:
#add fname
fname = "10180408_8"
info = {
    "l1_strength": -0.001,
    "l2_strength": 0.0,
    "lr": 0.01,
}
info["file_name"] = fname

args = ProtendArgs()
device = get_device()


train_data = TwilightDuo(n_samples=500, bias=0, transform=transforms.ToTensor())
train_loader = DataLoader(train_data, shuffle=True, batch_size=25, num_workers=1, pin_memory=True)
val_data = TwilightDuo(n_samples=1000, bias=0, transform=transforms.ToTensor())
val_loader = DataLoader(val_data, shuffle=False, batch_size=100, num_workers=1, pin_memory=True)

network = load_model(args, fname=fname).to(device)
loss_fn = nn.CrossEntropyLoss()

#try different lrs
optimizer = optim.Adam(network.classifier.parameters(), lr=0.1)

In [3]:
def log_info(info):
    max_len = max(len(arg_name) for arg_name in info)
    line = '*' * (max_len + 2 + 20)

    print(line)

    print("Arguments & Hyperparameters".center(max_len + 2 + 20))

    for arg_name, arg_value in info.items():
        print(f"{arg_name.ljust(max_len)}: {arg_value}")

    print(line)

log_info(info)


for i in range(50):
    logs = [f"Epoch {i + 1:02}/{50}", ]
    tr_acc, tr_loss = run_epoch(train_loader, network, loss_fn, optimizer, True, device)
    val_acc, val_loss = run_epoch(val_loader, network, loss_fn, optimizer, False, device)
    tr_log = f"Train acc {tr_acc:.4f} loss {tr_loss:.4f}"
    val_log = f"Val acc {val_acc:.4f} loss {val_loss:.4f}"
    logs.append(tr_log)
    logs.append(val_log)
    print(", ".join(logs))

*********************************
   Arguments & Hyperparameters   
l1_strength: -0.001
l2_strength: 0.0
lr         : 0.01
file_name  : 10180408_8
*********************************
Epoch 01/50, Train acc 0.5000 loss 1.2223, Val acc 0.5000 loss 0.3419
Epoch 02/50, Train acc 0.5080 loss 0.5530, Val acc 0.5000 loss 0.0639
Epoch 03/50, Train acc 0.5320 loss 0.4280, Val acc 0.5000 loss 0.2209
Epoch 04/50, Train acc 0.5520 loss 0.4135, Val acc 0.5150 loss 0.0493
Epoch 05/50, Train acc 0.4920 loss 0.4777, Val acc 0.5200 loss 0.2308
Epoch 06/50, Train acc 0.4900 loss 0.3906, Val acc 0.5140 loss 0.0510
Epoch 07/50, Train acc 0.5400 loss 0.3018, Val acc 0.5190 loss 0.0674
Epoch 08/50, Train acc 0.5200 loss 0.3707, Val acc 0.5190 loss 0.0787
Epoch 09/50, Train acc 0.5400 loss 0.3788, Val acc 0.5360 loss 0.0293
Epoch 10/50, Train acc 0.5360 loss 0.3258, Val acc 0.5060 loss 0.1314
Epoch 11/50, Train acc 0.4880 loss 0.3885, Val acc 0.5500 loss 0.0239
Epoch 12/50, Train acc 0.4960 loss 0.4541, Val ac